# Value Iteration Algorithms

In [2]:
import random
import numpy as np

In [3]:
# CONSTANTS
team_number = 92
arr = [1/2, 1, 2]
y = arr[team_number%3]
STEP_COST = -10/y

GAMMA = 0.999
DELTA = 0.001

In [4]:
X = 0
Y = 0
POSITION = "CENTER"

In [5]:
positions = {
    (0,0): "CENTER",
    (-1,0): "WEST",
    (1,0): "EAST",
    (0,1): "NORTH",
    (0,-1): "SOUTH"
}

coordinates = {
    "CENTER": (0,0),
    "WEST": (-1,0),
    "EAST": (1,0),
    "NORTH": (0,1),
    "SOUTH": (0,-1)
}

possible_actions = {
    "CENTER": ["UP", "DOWN", "LEFT", "RIGHT", "STAY", "SHOOT", "HIT"],
    "WEST": ["RIGHT", "STAY", "SHOOT"],
    "EAST": ["LEFT", "STAY", "SHOOT", "HIT"],
    "NORTH": ["DOWN", "STAY", "CRAFT"],
    "SOUTH": ["UP", "STAY", "GATHER"]
}
direction_tuple = ["CENTER", "WEST", "EAST", "NORTH", "SOUTH"]


In [6]:
utilities = np.zeros((5,3,4,2,5))
history = []
history.append(utilities)
# location, materials, arrows, monster attacc?, monster aliven'tn't

In [7]:
# MOVEMENT FUNCTIONS

def move_up(state):
    new_state = state.copy()
    
    old_coordinates = coordinates[direction_tuple[new_state[0]]]
    (x, y) = old_coordinates
    y += 1
    new_state[0] = (x, y)
    return new_state
    

def move_down(pos):
    new_state = state.copy()
    
    old_coordinates = coordinates[direction_tuple[new_state[0]]]
    (x, y) = old_coordinates
    y -= 1
    new_state[0] = (x, y)
    return new_state
    

def move_left(pos):
    new_state = state.copy()
    
    old_coordinates = coordinates[direction_tuple[new_state[0]]]
    (x, y) = old_coordinates
    x -= 1
    new_state[0] = (x, y)
    return new_state
    

def move_right(pos):
    new_state = state.copy()
    
    old_coordinates = coordinates[direction_tuple[new_state[0]]]
    (x, y) = old_coordinates
    x -= 1
    new_state[0] = (x, y)
    return new_state
    

def move_stay(pos):
    new_state = state.copy()
    return new_state
    

def move(action, state):
    switcher = {
        "UP": move_up,
        "DOWN": move_down,
        "LEFT": move_left,
        "RIGHT": move_right,
        "STAY": move_stay
    }
    
    return(switcher[action](state))

# move("UP", "hehe")

# print(Y)

In [8]:
## TRANSITION FUNCTIONS
class State:
    def __init__ (self, name, x, y, move_prob, shoot_prob = 0, craft_prob = 0, gather_prob = 0, hit_prob = 0):
        super().__init__()
        
        self.name = name
        self.x = x
        self.y = y
        self.move_prob = move_prob
        self.shoot_prob = shoot_prob
        self.shoot_damage = 25
        self.hit_prob = hit_prob
        self.hit_damage = 50
        self.craft_prob = craft_prob

In [9]:
## INDIANA CLASS
class Indiana:
    def __init__ (self):
        self.x = 0
        self.y = 0
        self.position = positions[(self.x, self.y)]
        self.arrows = 0
        self.material = 0
        self.health = 0
        
        self.north = State("NORTH", 0, 1, 0.85, craft_prob = {1: 0.5, 2: 0.35, 3: 0.15})
        self.south = State("SOUTH", 0, -1, 0.85, gather_prob = 0.75)
        self.east = State("EAST", 1, 0, 1, shoot_prob = 0.9, hit_prob = 0.2)
        self.west = State("WEST", 0, 1, 1, shoot_prob = 0.25)
        self.center = State("CENTER", 0, 0, 0.85, shoot_prob = 0.5, hit_prob = 0.1)
        
    def act(self):
        self.position = positions[(self.x, self.y)]
        
        cur_actions = possible_actions[self.position]
        
        best_action = cur_actions[0]
        best_utility = -999999
        
        this_utility = 0
        
        
#         for i in cur_actions:
            
            
        
    def move(self, direction):
        
        states
        prob = random.random()
#         if probability_movement[self.position] == "NORTH":
#             if prob <= 0.85:
                

In [ ]:
## MM CLASS
class Monster:
    def __init__ (self):
        self.health = 100
        self.ready_state = 0
        
    def update(self):
        
        
        prob = random.random()
        
        
        # DORMANT state
        if not self.ready_state:
            if prob <= 0.2:
                self.ready_state = 1
        
        # READY state
        else:
            if prob <= 0.5
                self.attack()
                self.ready_state = 0
                
    def attack(self):
        if indiana.position == "EAST" or indiana.position == "CENTER":
            indiana.arrows = 0
            self.health = min(self.health + 25, 100)
            indiana.reward -= -40            

In [10]:
def fml(state, action):
    location = direction_tuple[state[0]]
    
    ret = []
    
    if location == "NORTH":
        if action in ["DOWN", "STAY"]:
            new_state = move(action, state)
            fail_state = state.copy()
            fail_state[0] = 2
            ret = [[0.85, 0.15], [new_state, fail_state]]
        else:
            arrow_probs = [0.5, 0.35, 0.15]
            arrow1 = state.copy()
            arrow2 = state.copy()
            arrow3 = state.copy()
            arrow1[1] -= 1
            arrow2[1] -= 1
            arrow3[1] -= 1
            arrow1[2] = min(3, arrow1[2] + 1)
            arrow2[2] = min(3, arrow2[2] + 2)
            arrow3[2] = min(3, arrow3[2] + 3)
            
            ret = [arrow_probs, [arrow1, arrow2, arrow3]]
            
  
    elif location == "SOUTH":
        if action in ["UP", "STAY"]:
            new_state = move(action, state)
            fail_state = state.copy()
            fail_state[0] = 2
            ret = [[0.85, 0.15], [new_state, fail_state]]
        else:
            gather_probs = [0.75, 0.25]
            new_state = state.copy()
            fail_state = new_state.copy()
            new_state[1] = min(2, new_state[1] + 1)
            ret = [gather_probs, [new_state, fail_state]]
                
                
    elif location == "WEST":
        if action in ["RIGHT", "STAY"]:
            new_state = move(action, state)
            fail_state = state.copy()
            fail_state[0] = 2
            ret = [[1, 0], [new_state, fail_state]]
        else:
            shoot_probs = [0.25, 0.75]
            new_state = state.copy()
            new_state[2] -= 1
            fail_state = new_state.copy()
            new_state[4] -= 25
            ret = [shoot_probs, [new_state, fail_state]]
            
            
    elif location == "EAST":
        if action in ["LEFT", "STAY"]:
            new_state = move(action, state)
            fail_state = state.copy()
            fail_state[0] = 2
            ret = [[1, 0], [new_state, fail_state]]
        elif action == "SHOOT":
            shoot_probs = [0.9, 0.1]
            new_state = state.copy()
            new_state[2] -= 1
            fail_state = new_state.copy()
            new_state[4] -= 25
            ret = [shoot_probs, [new_state, fail_state]]
        else:
            hit_probs = [0.2, 0.8]
            new_state = state.copy()
            fail_state = new_state.copy()
            new_state[4] = max(0, new_state[4] - 50)
            ret = [hit_probs, [new_state, fail_state]]
            
            
    elif location == "CENTER":
        if action in ["UP", "DOWN", "RIGHT", "LEFT", "STAY"]:
            new_state = move(action, state)
            fail_state = state.copy()
            fail_state[0] = 2
            ret = [[0.85, 0.15], [new_state, fail_state]]
        elif action == "SHOOT":
            shoot_probs = [0.5, 0.5]
            new_state = state.copy()
            new_state[2] -= 1
            fail_state = new_state.copy()
            new_state[4] -= 25
            ret = [shoot_probs, [new_state, fail_state]]
        else:
            hit_probs = [0.1, 0.9]
            new_state = state.copy()
            fail_state = new_state.copy()
            new_state[4] = max(0, new_state[4] - 50)
            ret = [hit_probs, [new_state, fail_state]]
            
    # monster slep slep 
    if not state[3]:
        awake_prob = [0.2, 0.8]
        probabilities = ret[0]
        states = ret[1]
        
        new_probs = []
        new_states = []
        
        for i, prob in enumerate(probabilities):
            new_probs.append(prob*awake_prob[0])
            new_probs.append(prob*awake_prob[1])
            awake_state = states[i].copy()
            awake_state[3] = 1
            new_states.append(awake_state)
            new_states.append(state[i].copy())

    # monster wakey wakey   
    else:
        probabilities = ret[0]
        states = ret[1]
        
        new_probs = []
        new_states = []
        
        if location == "EAST" or "CENTER":
            attacc_prob = [0.5, 0.5]
            
            for idx, prob in enumerate(probabilities):
                new_probs.append(prob*attacc_prob[0])
                new_states.append(states[i].copy())
            new_probs.append(attacc_prob[1])
            
            og_state = state.copy()
            og_state[2] = 0 # arrows
            og_state[4] += 25 # helth
            og_state[3] = 0 # dormant state
            new_states.append(og_state)
            
        else:
            awake_prob = [0.5, 0.5]
            probabilities = ret[0]
            states = ret[1]

            new_probs = []
            new_states = []

            for i, prob in enumerate(probabilities):
                new_probs.append(prob*awake_prob[0])
                new_probs.append(prob*awake_prob[1])
                awake_state = states[i].copy()
                awake_state[3] = 0
                new_states.append(awake_state)
                new_states.append(state[i].copy())
            

In [12]:
## iteration loop

indiana = Indiana()
cur_error = 999999999


# while(cur_error > DELTA):
for state, val in np.ndenumerate(utilities):
    cur_actions = possible_actions[direction_tuple[state[0]]].copy()

    if 'SHOOT' in cur_actions and state[2] == 0:
        cur_actions.remove('SHOOT')
        
    
    if 'CRAFT' in cur_actions and state[1] == 0:
        cur_actions.remove('CRAFT')
        
    if state[4] == 0:
        cur_actions = ['NONE']
        
    
    
#     for action in cur_actions:
#         if state[0] not in [1, 2]:
            
        
    
#     print()
        

In [12]:
print(possible_actions)

{'CENTER': ['UP', 'DOWN', 'LEFT', 'RIGHT', 'STAY', 'SHOOT', 'HIT'], 'WEST': ['RIGHT', 'STAY', 'SHOOT'], 'EAST': ['LEFT', 'STAY', 'SHOOT', 'HIT'], 'NORTH': ['DOWN', 'STAY', 'CRAFT'], 'SOUTH': ['UP', 'STAY', 'GATHER']}
